In [3]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
import finbert.utils as tools

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
standard_dir = project_dir.parent.parent

# Prepare model

In [8]:
lm_path = project_dir/'models'/'language_model'/'finbertTRC2'
# classification_data_path= standard_dir/'Data'/'sentiment_data'

## Configure training parameters

In [12]:
# Clean the cl_path
try:
    shutil.rmtree(cl_path) 
except:
    pass

## load the pretrained language model trained on Reuters TRC2
bertmodel = AutoModelForSequenceClassification.from_pretrained(
    lm_path, cache_dir=None, num_labels=30)

Some weights of the model checkpoint at c:\Programming\MoneyLion\standard\advanced\finBERT_transaction_classification\models\language_model\finbertTRC2 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
S

In [ ]:
config = Config(
    data_dir=classification_data_path,
    bert_model=bertmodel,
    num_train_epochs=5,
    max_seq_length=64,
    train_batch_size=32,
    learning_rate=2e-5,
    output_mode="classification",
    warm_up_proportion=0.2,
    local_rank=-1,
    discriminate=True,
    gradual_unfreeze=True
)

In [ ]:
finbert = FinBert(config)

In [ ]:
finbert.prepare_model(label_list=[])